<a href="https://colab.research.google.com/github/Frasierzzz/Project/blob/main/highlight_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lib & Dataset

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("scitldr")

README.md:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

scitldr.py:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/7.56k [00:00<?, ?B/s]

The repository for scitldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scitldr.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'source_labels', 'rouge_scores', 'paper_id', 'target'],
        num_rows: 1992
    })
    test: Dataset({
        features: ['source', 'source_labels', 'rouge_scores', 'paper_id', 'target'],
        num_rows: 618
    })
    validation: Dataset({
        features: ['source', 'source_labels', 'rouge_scores', 'paper_id', 'target'],
        num_rows: 619
    })
})

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['validation']

In [ ]:
len(train_dataset['target'][:])

1992

# Wikification

In [ ]:
import json
import requests
import pandas as pd

In [ ]:
WIKIFIER_URL = "http://www.wikifier.org/annotate-article"
API_KEY = "pwdxjzlnmegtncwumufswitkbefgxb"

In [ ]:
CHUNK_SIZE = 15000
BULK_SIZE = 50
DF_IGNORE_VAL = 50
WORDS_IGNORE_VAL = 50

In [ ]:
ACCURACY_FIELD = u'rho'
TITLE_FIELD = u'title'
COSINE_FIELD = u'cosine'
PAGERANK_FIELD = u'pageRank'
WIKI_DATA_ID_FIELD = u'wikiDataItemId'
URL_FIELD = u'url'
STATUS_FIELD = u'status'
ANNOTATION_DATA_FIELD = u'annotation_data'

In [ ]:
SENTENCE_AGGREGATOR = " "
LEN_SENTENCE_AGGR = len(SENTENCE_AGGREGATOR)
SILENCE_INDICATORS = ["~silence~", "~SILENCE~", "~SIL", "[SILENCE]"]
HESITATION_INDICATORS = ["[hesitation]", "[HESITATION]"]
UNKNOWN_INDICATORS = ["<unk>", "[UNKNOWN]", "[unknown]"]
SPECIAL_TOKENS = set(SILENCE_INDICATORS + HESITATION_INDICATORS + UNKNOWN_INDICATORS)
ERROR_KEY = u'error'

In [ ]:
def get_clean_text(dataset):
    cleaned_list = []
    for example in dataset:
        cleaned_example = {key: text.replace(substr, "") if isinstance(text, str) else text
                           for key, text in example.items() for substr in SPECIAL_TOKENS}
        cleaned_list.append(cleaned_example)
    return cleaned_list

In [ ]:
cleaned_train_dataset = get_clean_text(train_dataset)
cleaned_test_dataset = get_clean_text(test_dataset)
cleaned_val_dataset = get_clean_text(val_dataset)

In [ ]:
print(len(cleaned_train_dataset),
     len(cleaned_test_dataset ),
     len(cleaned_val_dataset) )

1992 618 619


In [ ]:
def get_wikifier_wikify_response(text, api_key, df_ignore, words_ignore):
    params = {
        "text": text,
        "userKey": api_key,
        "nTopDfValuesToIgnore": df_ignore,
        "nWordsToIgnoreFromList": words_ignore
    }
    r = requests.post(WIKIFIER_URL, params)
    if r.status_code == 200:
        resp = json.loads(r.content)
        if ERROR_KEY in resp:
            raise ValueError("error in response : {}".format(resp[ERROR_KEY]))
        return resp
    else:
        raise ValueError("http status code 200 expected, got status code {} instead".format(r.status_code))

In [ ]:
def get_wikififier_concepts(resp, top_n=5):
    annotations = list(sorted([{TITLE_FIELD: ann[TITLE_FIELD],
                                URL_FIELD: ann[URL_FIELD],
                                COSINE_FIELD: ann[COSINE_FIELD],
                                PAGERANK_FIELD: ann[PAGERANK_FIELD],
                                WIKI_DATA_ID_FIELD: ann.get(WIKI_DATA_ID_FIELD)}
                               for ann in resp.get("annotations", [])],
                              key=lambda record: record[PAGERANK_FIELD], reverse=True))

    if top_n is not None:
        annotations = list(annotations)[:top_n]

    # Check if 'status' key exists before accessing it
    status = resp.get(STATUS_FIELD)
    if status is None:
        status = "unknown"  # Or handle it differently based on your needs

    return {
        ANNOTATION_DATA_FIELD: annotations,
        STATUS_FIELD: status
    }

In [ ]:
def wikify_text(text, api_key, top_n=5):
    try:
        response = get_wikifier_wikify_response(text, api_key, DF_IGNORE_VAL, WORDS_IGNORE_VAL)
        return get_wikififier_concepts(response, top_n)
    except Exception as e:
        return {"error": str(e)}

In [ ]:
import time
import random
from collections import Counter

def relate_topic(datadict, max_retries=3, delay=2):
    result = []
    for data in datadict:
        source = data['source']
        rouge_scores = data['rouge_scores']
        summary = data['target']

        top_source_indices = sorted(range(len(rouge_scores)), key=lambda i: rouge_scores[i], reverse=True)[:5]
        top_source_text = " ".join([source[i] for i in top_source_indices])  # รวม top 5 เป็นข้อความเดียว

        def safe_wikify(text, api_key, top_n=None):
            retries = 0
            while retries < max_retries:
                try:
                    result = wikify_text(text, api_key)
                    result['annotation_data'] = result['annotation_data'][:top_n] if top_n else result['annotation_data']
                    return result
                except Exception as e:
                    print(f"API Error: {e}. Retrying {retries + 1}/{max_retries}...")
                    time.sleep(delay + random.uniform(0, 1))
                    retries += 1
            return {"annotation_data": []}

        source_topics = safe_wikify(top_source_text, API_KEY, top_n=5)  # เรียก API แค่ครั้งเดียว
        source_titles = [item['title'] for item in source_topics['annotation_data']]
        print("Top 5 Source Titles:", source_titles)

        source_topics_ids = [item['wikiDataItemId'] for item in source_topics['annotation_data']]

        if isinstance(summary, list):
            summary_topics_list = [safe_wikify(s, API_KEY, top_n=3) for s in summary]
        else:
            summary_topics_list = [safe_wikify(summary, API_KEY, top_n=3)]

        summary_titles = [[item['title'] for item in topics['annotation_data']] for topics in summary_topics_list]
        print("Top 2 Target Titles:", summary_titles)

        final_topics = []
        for summary_topics in summary_topics_list:
            summary_ids = [item['wikiDataItemId'] for item in summary_topics['annotation_data']]
            intersection = list(set(source_topics_ids) & set(summary_ids))

            if intersection:
                count = Counter(intersection)
                most_common = count.most_common(1)[0][0]
                title = next(item['title'] for item in source_topics['annotation_data'] if item['wikiDataItemId'] == most_common)
                final_topics.append([title])
            else:
                final_topics.append([])

        result.append({
            "source": data.get("source", None),
            "source_labels": data.get("source_labels", None),
            "rouge_scores": data.get("rouge_scores", None),
            "paper_id": data.get("paper_id", None),
            "target": summary,
            "topic": final_topics
        })

    return result


In [ ]:
cleaned_train_dataset1 = cleaned_train_dataset[:498]
len(cleaned_train_dataset1)

498

In [ ]:
results_test = relate_topic(cleaned_train_dataset1[7:8])

Top 5 Source Titles: ['Loss function', 'Weightlifting at the Commonwealth Games', 'Logit', 'Deep learning', 'Gradient']
Top 2 Target Titles: [['Feature learning', 'Machine learning', 'Gradient']]


In [ ]:
response = get_wikifier_wikify_response('It is fundamental and challenging to train robust and accurate Deep Neural Networks (DNNs) when semantically abnormal examples exist.'
    , API_KEY, DF_IGNORE_VAL, WORDS_IGNORE_VAL)
get_wikififier_concepts(response, 5)

{'annotation_data': [{'title': 'Deep learning',
   'url': 'http://en.wikipedia.org/wiki/Deep_learning',
   'cosine': 0.1782035336792373,
   'pageRank': 0.0614478633391814,
   'wikiDataItemId': 'Q197536'},
  {'title': 'Semantics',
   'url': 'http://en.wikipedia.org/wiki/Semantics',
   'cosine': 0.02888627773688199,
   'pageRank': 0.04569060565672041,
   'wikiDataItemId': 'Q796212'},
  {'title': 'Artificial neural network',
   'url': 'http://en.wikipedia.org/wiki/Artificial_neural_network',
   'cosine': 0.1458037890177944,
   'pageRank': 0.03598837569559831,
   'wikiDataItemId': None},
  {'title': 'Neural network',
   'url': 'http://en.wikipedia.org/wiki/Neural_network',
   'cosine': 0.1302744432854702,
   'pageRank': 0.02750361546679042,
   'wikiDataItemId': 'Q12811862'},
  {'title': 'Accuracy and precision',
   'url': 'http://en.wikipedia.org/wiki/Accuracy_and_precision',
   'cosine': 0.04239203344415145,
   'pageRank': 0.02214774523293677,
   'wikiDataItemId': 'Q272035'}],
 'status': 

In [ ]:
results_test

[{'source': ['It is fundamental and challenging to train robust and accurate Deep Neural Networks (DNNs) when semantically abnormal examples exist.',
   'Although great progress has been made, there is still one crucial research question which is not thoroughly explored yet: What training examples should be focused and how much more should they be emphasised to achieve robust learning?',
   'In this work, we study this question and propose gradient rescaling (GR) to solve it.',
   'GR modifies the magnitude of logit vector’s gradient to emphasise on relatively easier training data points when noise becomes more severe, which functions as explicit emphasis regularisation to improve the generalisation performance of DNNs.',
   'Apart from regularisation, we connect GR to examples weighting and designing robust loss functions.',
   'We empirically demonstrate that GR is highly anomaly-robust and outperforms the state-of-the-art by a large margin, e.g., increasing 7% on CIFAR100 with 40% n

In [ ]:
import json
from google.colab import files

json_data = json.dumps(results_test , indent=4)

# Save JSON data to a file
with open('h2_train1.json', 'w') as f:
  f.write(json_data)

# Download the file
files.download('h2_train1.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>